In [3]:
from crewai_tools import ScrapeWebsiteTool, FileWriterTool, TXTSearchTool
import requests
from crewai import Agent, Task, Crew, Process
import os
from crewai_tools import TXTSearchTool


In [2]:

# Initialize the tool, potentially passing the session
scrape_tool = ScrapeWebsiteTool(website_url='https://en.wikipedia.org/wiki/Artificial_intelligence')

# Extract the text
text = scrape_tool.run()
print(text[:100])

Using Tool: Read website content
Artificial intelligence - Wikipedia
Jump to content
Main menu
Main menu
move to sidebar
hide
		Navig


In [20]:
# Initialize the tool
file_writer_tool = FileWriterTool()

# Write content to a file in a specified directory
result = file_writer_tool._run(filename='ai.txt', content = text, directory = '', overwrite=True)
print(result)

Content successfully written to ai.txt


In [21]:

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Initialize the tool with a specific text file, so the agent can search within the given text file's content
# uses chromadb to chunk and vectorize data
search_tool = TXTSearchTool(txt='ai.txt')

In [65]:
from crewai import Agent, Task, Crew

data_analyst = Agent(
    role='Educator',
    goal=f'Based on the context provided, answer the question - What is Natural Language Processing?',
    backstory='You are a data expert',
    verbose=True,
    allow_delegation=False,
    tools=[search_tool]
)

test_task = Task(
    description="Understand the topic of Natural Language Processing and give the correct response",
    agent=data_analyst,
    expected_output='Give a correct response'
)

crew = Crew(
    tasks=[test_task],
    process=Process.sequential,
)

output = crew.kickoff()

# Agent: Educator
## Task: Understand the topic of Natural Language Processing and give the correct response


# Agent: Educator
## Using tool: Search a txt's content
## Tool Input: 
"{\"search_query\": \"Natural Language Processing\"}"
## Tool Output: 
Relevant Content:
Natural language processing (NLP)[50] allows programs to read, write and communicate in human languages such as English. Specific problems include speech recognition, speech synthesis, machine translation, information extraction, information retrieval and question answering.[51]
Early work, based on Noam Chomsky's generative grammar and semantic networks, had difficulty with word-sense disambiguation[f] unless restricted to small domains called "micro-worlds" (due to the common sense knowledge problem[29]). Margaret Masterman believed that it was meaning and not grammar that was the key to understanding languages, and that thesauri and not dictionaries should be the basis of computational language structure.
Modern dee

In [66]:
print(output.raw)

Natural language processing (NLP) allows programs to read, write, and communicate in human languages such as English. Specific problems include speech recognition, speech synthesis, machine translation, information extraction, information retrieval, and question answering. Early work, based on Noam Chomsky's generative grammar and semantic networks, had difficulty with word-sense disambiguation unless restricted to small domains called "micro-worlds" (due to the common sense knowledge problem). Margaret Masterman believed that it was meaning and not grammar that was the key to understanding languages, and that thesauri and not dictionaries should be the basis of computational language structure.

Modern deep learning techniques for NLP include word embedding (representing words, typically as vectors encoding their meaning), transformers (a deep learning architecture using an attention mechanism), and others. In 2019, generative pre-trained transformer (or "GPT") language models began t

In [67]:
from langchain_openai import ChatOpenAI

test_task = Task(
    description="Understand the topic of Natural Language Processing and give the correct response",
    # agent=data_analyst,  # commenting out the agent now, letting the manager decide
    expected_output='Give a correct response'
)
calculator_agent = Agent(
    role='Calculator',
    goal=f'You calculate things',
    backstory='You love math',
    verbose=True,
    allow_delegation=False,
    tools=[]
)

math_task = Task(
    description="Tell me what 123 * 34 is",
    expected_output='Calculate this'
)


crew = Crew(
    tasks=[math_task, test_task],
    agents=[calculator_agent, data_analyst],
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7),
    verbose=True
)

output = crew.kickoff()

# Agent: Crew Manager
## Task: Understand the topic of Natural Language Processing and give the correct response


# Agent: Crew Manager
## Thought: Thought: I need to delegate the task of understanding the topic of Natural Language Processing and providing the correct response to the Educator, as they are skilled in explaining complex topics in an understandable manner.
## Using tool: Delegate work to coworker
## Tool Input: 
"{\"task\": \"Understand the topic of Natural Language Processing and provide a correct response\", \"context\": \"I need a detailed explanation of Natural Language Processing and its key components, applications, and importance in today's technological landscape.\"}"
## Tool Output: 

Error executing tool. coworker mentioned not found, it must be one of the following options:
- calculator
- educator

# Agent: Calculator
## Task: Understand the topic of Natural Language Processing and provide a correct response


# Agent: Calculator
## Final Answer: 
Natural Lang

In [68]:
for output in output.tasks_output:
    print(output)
    print('----')

Natural Language Processing (NLP) is an essential area of artificial intelligence that enables machines to understand and interact with human language. Through key components such as tokenization, morphological analysis, part-of-speech tagging, named entity recognition, parsing, sentiment analysis, machine translation, and speech technologies, NLP powers various applications like chatbots, search engines, social media monitoring, and content recommendations. Its importance is underscored by its ability to analyze vast amounts of data, facilitate real-time user interaction, and contribute to the advancement of automated systems, making it a cornerstone in today's technological advancements.
----
The result of 123 multiplied by 34 is 4182.
----


In [70]:
output.raw

'The result of 123 multiplied by 34 is 4182.'

# LangChain + CrewAI

In [ ]:
!pip install -q langchain-community duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 31.9 MB/s eta 0:00:00


In [ ]:
import os

from langchain_community.tools import DuckDuckGoSearchRun  # Initialize the tool
search_ddg_tool = DuckDuckGoSearchRun()

In [ ]:
search_ddg_tool.run('capital of Turkey')

'Ankara is the national capital of Turkey, located in the northwestern part of the country. It has a rich and diverse history, dating back to the Stone Age, and is home to many cultural and historical attractions, such as the Atatürk Mausoleum and the Museum of Anatolian Civilizations. Learn about Turkey, a country in Western Asia and Southeastern Europe, with a rich and diverse culture and history. The capital of Turkey is Ankara, and the largest city is Istanbul. Learn about Turkey, a country that occupies a unique geographic position, lying partly in Asia and partly in Europe. The capital of Turkey is Ankara, and its largest city and seaport is Istanbul. Istanbul, largest city and principal seaport of Turkey. Historically known as Byzantium and then Constantinople, it was the capital of the Byzantine Empire and the Ottoman Empire. Istanbul straddles the Bosporus strait, one of two waterways that separates the European and Asian parts of Turkey. Learn about Ankara, the vibrant capita

In [ ]:
#
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
  verbose=True,
  allow_delegation=False,
  tools=[search_ddg_tool]
)

writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=True
)

In [ ]:
# Create tasks for your agents
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
  expected_output="Full analysis report in bullet points",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
  expected_output="Full blog post of at least 4 paragraphs",
  agent=writer
)

In [ ]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  process=Process.sequential,
)

In [ ]:
# writes tries to delegate to research analyst. WRONG!
result = crew.kickoff()

print("-----------------------------")
print(result)


# Agent: Senior Research Analyst
## Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.


# Agent: Senior Research Analyst
## Thought: I need to gather information on the latest advancements in AI for the year 2024, focusing on key trends and breakthrough technologies. I will use the available tool to search for relevant current information.
## Using tool: duckduckgo_search
## Tool Input: 
"{\"query\": \"latest advancements in AI 2024 trends technologies breakthroughs\"}"
## Tool Output: 
In 2024, generative AI might actually become useful for the regular, non-tech person, and we are going to see more people tinkering with a million little AI models. State-of-the-art AI models ... When OpenAI launched a free web app called ChatGPT in November 2022, nobody knew what was coming. But that low-key release changed everything. By January, ChatGPT had become the fastest-growing ... H

KeyboardInterrupt: 

In [ ]:
# from langchain_openai import ChatOpenAI

researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
  verbose=True,
  allow_delegation=False,
  tools=[search_ddg_tool],
#   llm=ChatOpenAI(model_name="gpt-4o-mini", temperature=0.4)
)

writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=False
)

In [ ]:
# Create tasks for your agents
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
  expected_output="Full analysis report in bullet points",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
  expected_output="Full blog post of at least 4 paragraphs",
  agent=writer
)

In [ ]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  process=Process.sequential,
)
result = crew.kickoff()
print("-----------------------------")
print(result)

# Agent: Senior Research Analyst
## Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.


# Agent: Senior Research Analyst
## Thought: I need to search for the latest advancements in AI in 2024 to gather comprehensive insights about key trends, breakthrough technologies, and their potential industry impacts.
## Using tool: duckduckgo_search
## Tool Input: 
"{\"name\": \"duckduckgo_search\", \"description\": \"A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.\", \"args_schema\": {}, \"return_direct\": false, \"verbose\": false, \"tags\": null, \"metadata\": null, \"handle_tool_error\": false, \"handle_validation_error\": false, \"response_format\": \"content\", \"api_wrapper\": {}}"
## Tool Output: 
Error: the Action Input is not a valid key, value dictionary.


# Agent: Senior Research Analyst

In [ ]:
task1.output.raw

'- **Expansion of Generative AI Usage**:\n  - In 2024, 65% of organizations are regularly using generative AI, a significant increase from previous years.\n  - The focus has shifted from experimenting with generative AI to deriving tangible business value from it.\n\n- **Growth Rate Projections**:\n  - AI is anticipated to grow at an annual rate of 36.6% from 2023 to 2030, indicating strong market demand and potential for innovation.\n\n- **Accessibility to Non-Tech Users**:\n  - Generative AI is becoming more user-friendly, enabling regular users to engage with and utilize various AI models in their daily tasks.\n\n- **Improved Autonomous Chatbots**:\n  - Autonomous chatbots are evolving with smarter algorithms, enhancing customer service capabilities, and reducing response times.\n\n- **AI Tinkering and Model Development**:\n  - An increasing number of users are experimenting with developing or utilizing a multitude of small-scale AI models to meet specific needs.\n\n- **Diverse Appl

In [ ]:
task2.output.raw

"**The Future is Now: 2024’s Dazzling AI Advancements**\n\nWelcome to 2024, where artificial intelligence is not just another buzzword but a dynamic force reshaping the business landscape. In a world where 65% of organizations are now actively harnessing the power of generative AI, we're witnessing a monumental shift. Gone are the days of merely experimenting with AI; companies are focusing on extracting real-world value from it. From automating mundane tasks to generating insightful data-driven decisions, generative AI is becoming the backbone of innovation, driving efficiency, and giving businesses a competitive edge like never before.\n\nWhat's even more exhilarating is the staggering growth rate projected for AI. Anticipated to rise at an amazing annual rate of 36.6% from 2023 to 2030, AI is not just a passing trend; it’s a burgeoning market demand. With increasing accessibility, even those without a tech background are diving into AI. User-friendly platforms now allow everyone to 

In [ ]:
# use langhcain chatllm
from langchain_openai import ChatOpenAI

researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
  verbose=True,
  allow_delegation=False,
  tools=[search_ddg_tool],
  llm=ChatOpenAI(model_name="gpt-4o-mini", temperature=0.4)
)

In [ ]:
!pip install -q google-search-results

  Preparing metadata (setup.py) ... done


In [ ]:
import json
import os
import requests
from crewai import Agent, Task
from langchain.tools import tool
from serpapi import GoogleSearch
from crewai_tools import BaseTool

class SearchTool(BaseTool):
    name: str = "Google Search"
    description: str = """Useful to search the internet about a given topic and return relevant results."""

    def _run(self, query: str, top_k: int = 3) -> str:
        params: dict = {
            "engine": "google",
            "google_domain": "google.com",
            "gl": "us",
            "hl": "en",
            "q": query,
            "api_key": userdata.get("SERP_API_KEY"),
        }

        search = GoogleSearch(params)
        response = search.get_dict()
        # check if there is an organic results key, don't include sponsor results
        if 'organic_results' not in response:
            return "Sorry, I couldn't find anything about that, there could be an error with you serper api key."
        else:
            results = response['organic_results']
            string = []
            for result in results[:top_k]:
                try:
                    string.append('\n'.join([
                        f"Title: {result['title']}", f"Link: {result['link']}",
                        f"Snippet: {result['snippet']}", "\n-----------------"
                    ]))
                except KeyError:
                    next

            return '\n'.join(string)

In [ ]:
SearchTool().description # where I specifically describe how to use the tool

"Google Search(query: 'string', top_k: 'integer') - Useful to search the internet about a given topic and return relevant results. "

In [ ]:
SearchTool().run(query="Who is Sinan Ozdemir?")

Using Tool: Google Search


"Title: Sinan Ozdemir - LoopGenius\nLink: https://www.linkedin.com/in/sinan-ozdemir\nSnippet: Specialized in areas including time management as well as mathematics and computer science logistics.\n\n-----------------\nTitle: Sinan Ozdemir\nLink: https://sinanozdemir.ai/\nSnippet: Hi! My name is Sinan Ozdemir. I'm an AI expert, consultant, author, and founder. The author of Quick Start Guide to LLMs (companion Github) and the AI ...\n\n-----------------\nTitle: Sinan Ozdemir\nLink: https://odsc.com/blog/speaker/sinan-ozdemir/\nSnippet: Sinan Ozdemir is a mathematician, data scientist, NLP expert, lecturer, and accomplished author. He is currently applying my extensive knowledge and experience ...\n\n-----------------"

In [ ]:
researcher_agent = Agent(
    role='Researcher',
    goal='Look up information on the internet and scrape the websites for relevant information',
    backstory="""You have a knack for dissecting complex data and presenting actionable insights.""",
    tools=[
        SearchTool()
    ],
    verbose=True
)
sample_lookup_task = Task(
  description="""Lookup Sinan Ozdemir""",
  expected_output="A lookup report of a query",
  agent=researcher_agent
)

crew = Crew(
    tasks=[sample_lookup_task],
    agents=[researcher_agent],
    process=Process.sequential,
    verbose=True
)

result = crew.kickoff()

# Agent: Researcher
## Task: Lookup Sinan Ozdemir


# Agent: Researcher
## Thought: I need to search for information about Sinan Ozdemir to create a comprehensive lookup report.
## Using tool: Google Search
## Tool Input: 
"{\"query\": \"Sinan Ozdemir\", \"top_k\": 3}"
## Tool Output: 
Title: Sinan Ozdemir - LoopGenius
Link: https://www.linkedin.com/in/sinan-ozdemir
Snippet: Specialized in areas including time management as well as mathematics and computer science logistics.

-----------------
Title: Sinan Ozdemir
Link: https://sinanozdemir.ai/
Snippet: Hi! My name is Sinan Ozdemir. I'm an AI expert, consultant, author, and founder. The author of Quick Start Guide to LLMs (companion Github) and the AI ...

-----------------
Title: Prof_OZ - Sinan Ozdemir
Link: https://twitter.com/prof_oz
Snippet: NLP + Gen AI Expert / LLM whisperer AI Author Founder @Aikylie (acquired) Contributor on @Forbes Fellow @YCombinator.

-----------------


# Agent: Researcher
## Final Answer: 
### Lookup Repo

In [ ]:
print(result)

### Lookup Report: Sinan Ozdemir

#### 1. Professional Profile
- **Name:** Sinan Ozdemir
- **LinkedIn Profile:** [Sinan Ozdemir](https://www.linkedin.com/in/sinan-ozdemir)
  - **Specializations:** Expertise in time management, mathematics, and computer science logistics.

#### 2. Personal Website
- **Website:** [sinanozdemir.ai](https://sinanozdemir.ai/)
  - **About:** Sinan Ozdemir identifies himself as an AI expert, consultant, author, and founder. He is the author of the "Quick Start Guide to LLMs." The website serves as a platform for sharing knowledge and resources related to artificial intelligence.

#### 3. Social Media Presence
- **Twitter:** [Prof_OZ](https://twitter.com/prof_oz)
  - **Expertise:** He is recognized as a Natural Language Processing (NLP) and Generative AI expert, as well as a 'LLM whisperer.' He has authored works in the AI space and is the founder of Aikylie, a company that has been acquired. Additionally, he is a contributor for Forbes and a fellow at Y Combi

In [ ]:
print("""You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: Delegate work to coworker(task: str, context: str, coworker: Optional[str] = None, **kwargs)
Tool Description: Delegate a specific task to one of the following coworkers: Task Delegator
The input to this tool should be the coworker, the task you want them to do, and ALL necessary context to execute the task, they know nothing about the task, so share absolute everything you know, don't reference things but instead explain them.
Tool Arguments: {'name': FieldInfo(annotation=str, required=True), 'description': FieldInfo(annotation=str, required=False, default=''), 'args_schema': FieldInfo(annotation=type[BaseModel], required=True, description='The tool schema.', metadata=[SkipValidation()]), 'return_direct': FieldInfo(annotation=bool, required=False, default=False), 'verbose': FieldInfo(annotation=bool, required=False, default=False), 'callbacks': FieldInfo(annotation=Union[list[BaseCallbackHandler], BaseCallbackManager, NoneType], required=False, default=None, exclude=True), 'callback_manager': FieldInfo(annotation=Union[BaseCallbackManager, NoneType], required=False, default=None, description='.. deprecated:: 0.1.7 Use :meth:`~callbacks` instead.\n\nCallback manager to add to the run trace.', exclude=True), 'tags': FieldInfo(annotation=Union[list[str], NoneType], required=False, default=None), 'metadata': FieldInfo(annotation=Union[dict[str, Any], NoneType], required=False, default=None), 'handle_tool_error': FieldInfo(annotation=Union[bool, str, Callable[list, str], NoneType], required=False, default=False), 'handle_validation_error': FieldInfo(annotation=Union[bool, str, Callable[list, str], NoneType], required=False, default=False), 'response_format': FieldInfo(annotation=Literal['content', 'content_and_artifact'], required=False, default='content'), 'func': FieldInfo(annotation=Union[Callable[..., Any], NoneType], required=False, default=None), 'coroutine': FieldInfo(annotation=Union[Callable[..., Awaitable[Any]], NoneType], required=False, default=None)}
Tool Name: Ask question to coworker(question: str, context: str, coworker: Optional[str] = None, **kwargs)
Tool Description: Ask a specific question to one of the following coworkers: Task Delegator
The input to this tool should be the coworker, the question you have for them, and ALL necessary context to ask the question properly, they know nothing about the question, so share absolute everything you know, don't reference things but instead explain them.
Tool Arguments: {'name': FieldInfo(annotation=str, required=True), 'description': FieldInfo(annotation=str, required=False, default=''), 'args_schema': FieldInfo(annotation=type[BaseModel], required=True, description='The tool schema.', metadata=[SkipValidation()]), 'return_direct': FieldInfo(annotation=bool, required=False, default=False), 'verbose': FieldInfo(annotation=bool, required=False, default=False), 'callbacks': FieldInfo(annotation=Union[list[BaseCallbackHandler], BaseCallbackManager, NoneType], required=False, default=None, exclude=True), 'callback_manager': FieldInfo(annotation=Union[BaseCallbackManager, NoneType], required=False, default=None, description='.. deprecated:: 0.1.7 Use :meth:`~callbacks` instead.\n\nCallback manager to add to the run trace.', exclude=True), 'tags': FieldInfo(annotation=Union[list[str], NoneType], required=False, default=None), 'metadata': FieldInfo(annotation=Union[dict[str, Any], NoneType], required=False, default=None), 'handle_tool_error': FieldInfo(annotation=Union[bool, str, Callable[list, str], NoneType], required=False, default=False), 'handle_validation_error': FieldInfo(annotation=Union[bool, str, Callable[list, str], NoneType], required=False, default=False), 'response_format': FieldInfo(annotation=Literal['content', 'content_and_artifact'], required=False, default='content'), 'func': FieldInfo(annotation=Union[Callable[..., Any], NoneType], required=False, default=None), 'coroutine': FieldInfo(annotation=Union[Callable[..., Awaitable[Any]], NoneType], required=False, default=None)}

Use the following format:

Thought: you should always think about what to do
Action: the action to take, only one name of [Delegate work to coworker, Ask question to coworker], just the name, exactly as it's written.
Action Input: the input to the action, just a simple python dictionary, enclosed in curly braces, using " to wrap keys and values.
Observation: the result of the action

Once all necessary information is gathered:

Thought: I now know the final answer
Final Answer: the final answer to the original input question""")

You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: Delegate work to coworker(task: str, context: str, coworker: Optional[str] = None, **kwargs)
Tool Description: Delegate a specific task to one of the following coworkers: Task Delegator
The input to this tool should be the coworker, the task you want them to do, and ALL necessary context to execute the task, they know nothing about the task, so share absolute everything you know, don't reference things but instead explain them.
Tool Arguments: {'name': FieldInfo(annotation=str, required=True), 'description': FieldInfo(annotation=str, required=False, default=''), 'args_schema': FieldInfo(annotation=type[BaseModel], required=True, description='The tool schema.', metadata=[SkipValidation()]), 'return_direct': FieldInfo(annotation=bool, required=False, default=False), 'verbose': FieldInfo(annotation=bool, required=False, default=False), 'callbacks': FieldInfo(annotation=Union[l

## Trip Advisor

In [ ]:
from crewai import Agent

class TripAgents():

  def local_expert(self):
    return Agent(
        role='Local Expert at this city',
        goal='Provide 2-3 insights about the selected city',
        backstory="""A knowledgeable local guide with extensive information
        about the city, it's attractions and customs""",
        tools=[
            SearchTool(),
            scrape_tool
        ],
        verbose=True)

  def travel_concierge(self):
    return Agent(
        role='Amazing Travel Concierge',
        goal="""Create the most amazing travel itineraries with budget and
        packing suggestions for the city""",
        backstory="""Specialist in travel planning and logistics with
        decades of experience""",
        tools=[],
        verbose=True)

In [ ]:
from crewai import Task
from textwrap import dedent
from datetime import date


class TripTasks():

  def gather_task(self, agent, origin, city, interests, range):
    return Task(description= dedent(f"""
        I want you to create a comprehensive city guide.
        This guide should provide a thorough overview of what
        the city has to offer, including hidden gems, cultural
        hotspots, and must-visit landmarks.

        Trip Date: {range}
        Traveling from: {origin}
        Traveling to: {city}
        Traveler Interests: {interests}
        """),

            expected_output=dedent(f"""
        The final answer must be a comprehensive city guide,
        rich in cultural insights and practical tips,
        tailored to enhance the travel experience.

        """),
                agent=agent)

  def plan_task(self, agent, origin, city, interests, range):
    return Task(description=dedent(f"""
        Expand this guide into a a full 7-day travel
        itinerary with detailed per-day plans, including
        weather forecasts, places to eat, packing suggestions,
        and a budget breakdown.

        You MUST suggest actual places to visit, actual hotels
        to stay and actual restaurants to go to.

        Trip Date: {range}
        Traveling from: {origin}
        Traveling to: {city}
        Traveler Interests: {interests}
      """),
            expected_output=dedent(f"""
        Your final answer MUST be a complete expanded travel plan,
        formatted as markdown, encompassing a daily schedule,
        anticipated weather conditions, recommended clothing and
        items to pack, and a detailed budget, ensuring THE BEST
        TRIP EVER, Be specific and give it a reason why you picked
        # up each place, what make them special!
        """),
                agent=agent)

In [ ]:
class TripCrew:

  def __init__(self, origin, city, date_range, interests):
    self.city = city
    self.origin = origin
    self.interests = interests
    self.date_range = date_range

  def run(self):
    agents = TripAgents()
    tasks = TripTasks()

    gather_task = tasks.gather_task(
      agents.local_expert(),
      self.origin,
      self.city,
      self.interests,
      self.date_range
    )
    plan_task = tasks.plan_task(
      agents.travel_concierge(),
      self.origin,
      self.city,
      self.interests,
      self.date_range
    )

    # Initiate Crew
    crew = Crew(
      tasks=[gather_task, plan_task],
      process=Process.sequential,
      verbose=True
    )

    result = crew.kickoff()
    return result

In [ ]:
trip_crew = TripCrew('SF', "Paris", 'January 2025', 'art')
result = trip_crew.run()

# Agent: Local Expert at this city
## Task: 
I want you to create a comprehensive city guide. 
This guide should provide a thorough overview of what 
the city has to offer, including hidden gems, cultural
hotspots, and must-visit landmarks.

Trip Date: January 2025
Traveling from: SF
Traveling to: Paris
Traveler Interests: art



# Agent: Local Expert at this city
## Thought: I need to gather detailed information about Paris, focusing on art-related attractions, cultural hotspots, hidden gems, and must-visit landmarks to create a comprehensive city guide tailored for a trip in January 2025.
## Using tool: Google Search
## Tool Input: 
"{\"query\": \"Paris art attractions cultural hotspots hidden gems travel guide\", \"top_k\": 5}"
## Tool Output: 
Title: 48 Hours in Paris |Guide Hidden Gems
Link: https://www.dutchbloggeronthemove.com/48-hours-in-paris-guide-hidden-gems/
Snippet: From the cafés, restaurants, and bars, to my favorite activities, read on to see how you can make the most o

In [ ]:
print(result)

# **7-Day Travel Itinerary for Art Lovers in Paris (January 2025)**

## **Overview**
Prepare for a week of artistic indulgence in the City of Light! Experience world-class museums, vibrant neighborhoods, and culinary delights, all set against the backdrop of Paris's winter beauty.

### **Weather Forecast for January**
- Average temperatures: 3-7°C (37-45°F)
- Expected precipitation: Light rain possible
- **Packing Suggestions**: 
  - Warm layers: sweaters, long-sleeve shirts, and thermal underwear
  - Waterproof jacket and umbrella
  - Comfortable walking shoes
  - Scarf, gloves, and a hat for added warmth
  - Art journal or sketchbook for impromptu doodles

### **Budget Overview**
- **Flights**: ~$700 roundtrip from SF to Paris (booked in advance)
- **Accommodation**: ~$180 per night for 6 nights → $1,080
- **Daily Expenses**:
  - Food: ~$50/day → $350 for 7 days
  - Transport (Navigo pass): ~$30
  - Museum entry fees: ~$50
  - Miscellaneous (shopping, snacks): ~$100
- **Total Estimat

In [ ]:
if __name__ == "__main__":
  print("## Welcome to Trip Planner Crew")
  print('-——————————')
  location = input(
    dedent("""
      From where will you be traveling from?
    """))
  city = input(
    dedent("""
      Where do you want to go?
    """))
  date_range = input(
    dedent("""
      What is the date range you are interested in traveling?
    """))
  interests = input(
    dedent("""
      What are some of your high level interests and hobbies?
    """))

  trip_crew = TripCrew(location, city, date_range, interests)
  result = trip_crew.run()
  print("\n\n########################")
  print("## Here is you Trip Plan")
  print("########################\n")
  print(result)

## Welcome to Trip Planner Crew
-——————————

From where will you be traveling from?
sf

What are the cities options you are interested in visiting?
paris

What is the date range you are interested in traveling?
january

What are some of your high level interests and hobbies?
art


TypeError: TripTasks.gather_task() takes 5 positional arguments but 6 were given

In [ ]:
p